In [51]:
# Cell 1 — imports
import os
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve

print("Libraries imported. CWD:", os.getcwd())

Libraries imported. CWD: c:\Users\aayus\Desktop\McMaster\Term 1\DL\Project\AI Incident Detection System\notebooks


In [52]:
# Cell 2 — file paths (update if needed)
DATA_CSV = "../results/data_risk_scores.csv"
CODE_CSV = "../results/code_risk_scores.csv"

for p in (DATA_CSV, CODE_CSV):
    if not os.path.exists(p):
        raise FileNotFoundError(f"Expected file '{p}' in working directory but it was not found. Please upload it.")
print("Both CSV files found.")

Both CSV files found.


In [53]:
# Cell 3 — load CSVs
df_data = pd.read_csv(DATA_CSV)
df_code = pd.read_csv(CODE_CSV)

print("data rows:", len(df_data), "columns:", df_data.columns.tolist())
print("code rows:", len(df_code), "columns:", df_code.columns.tolist())

data rows: 13204 columns: ['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount', 'defects', 'reconstruction_mse', 'data_risk_score']
code rows: 5000 columns: ['code_risk_score']


In [54]:
# Cell 4 — validate columns and aggregate code_risk_score
if "data_risk_score" not in df_data.columns:
    matches = [c for c in df_data.columns if c.lower() == "data_risk_score"]
    if matches:
        df_data = df_data.rename(columns={matches[0]: "data_risk_score"})
    else:
        raise ValueError("data_risk_score column not found in data CSV.")

if "code_risk_score" not in df_code.columns:
    matches = [c for c in df_code.columns if c.lower() == "code_risk_score"]
    if matches:
        df_code = df_code.rename(columns={matches[0]: "code_risk_score"})
    else:
        raise ValueError("code_risk_score column not found in code CSV.")

# aggregate code_risk_score by mean and broadcast
code_scalar = float(df_code["code_risk_score"].astype(float).mean())
df_data = df_data.copy()
df_data["code_risk_score"] = code_scalar

print(f"Broadcasted code_risk_score mean={code_scalar:.6f} to all {len(df_data)} rows.")

Broadcasted code_risk_score mean=0.204284 to all 13204 rows.


In [55]:
# Cell 5 — prepare training data (label column 'defects' expected)
LABEL_COL = "defects"
if LABEL_COL not in df_data.columns:
    raise ValueError(f"Label column '{LABEL_COL}' not found in data CSV. Provide labels to train Model-C.")

df_model = df_data[["data_risk_score","code_risk_score", LABEL_COL]].dropna().rename(columns={LABEL_COL: "label"})
df_model["label"] = df_model["label"].astype(int)

X = df_model[["data_risk_score","code_risk_score"]].astype(float).values
y = df_model["label"].values

print("Rows available for training:", len(df_model))

Rows available for training: 13204


In [56]:
# Cell 6 — train model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

pipe = Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=2000, solver="liblinear"))])
param_grid = {"clf__C": [0.01,0.1,1.0,10.0], "clf__class_weight": [None, "balanced"]}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring="average_precision", n_jobs=-1)
grid.fit(X_train, y_train)
best_clf = grid.best_estimator_
print("Best params:", grid.best_params_)

Best params: {'clf__C': 0.01, 'clf__class_weight': None}


In [57]:
# Cell 7 — calibrate and compute all probabilities
calibrator = CalibratedClassifierCV(best_clf, cv="prefit", method="sigmoid")
calibrator.fit(X_train, y_train)

X_all = df_data[["data_risk_score","code_risk_score"]].astype(float).values
probs_all = calibrator.predict_proba(X_all)[:,1]

# compute average
final_avg_score = float(probs_all.mean())
final_avg_score

c:\Users\aayus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


0.15944272539306323

In [58]:
# Cell 8 — fixed thresholds and final print (ONLY ONE LINE)
# Fixed thresholds per your request:
thresholds_fixed = {"low_medium": 0.30, "medium_high": 0.60}

def format_incident_line_fixed(score, thresholds):
    score_rounded = round(float(score), 4)
    if score >= thresholds["medium_high"]:
        bucket = "HIGH"
    elif score >= thresholds["low_medium"]:
        bucket = "MEDIUM"
    else:
        bucket = "LOW"
    return f"Final Incident Score (average)={score_rounded:.4f} -> Probability of Incident={bucket}"

print(format_incident_line_fixed(final_avg_score, thresholds_fixed))

Final Incident Score (average)=0.1594 -> Probability of Incident=LOW
